# parser

In [210]:
import pandas as pd
import re, requests, time
from bs4 import BeautifulSoup
from tqdm import tqdm

PATH_TO_LOCATION_WITH_PROGRAMS = '/home/antek/Desktop/parse_cvk_2019/programs/'

def get_links_for_okrugs():
    request = requests.get('https://www.cvk.gov.ua/pls/vnd2019/wp032pt001f01=919.html')
    request.encoding = 'UTF-8'
    soup = BeautifulSoup(request.content, 'lxml')
    tds = soup.find_all('td', {'class':'td2'})
    okrugs = ['https://www.cvk.gov.ua/pls/vnd2019/' + td.a['href'] for td in tds if td.a is not None]
    return okrugs

def parse_candidates_info(url):
    request = requests.get(url)
    request.encoding = 'UTF-8'
    soup_okrug = BeautifulSoup(request.content, 'lxml')
    cands_table = soup_okrug.find_all('table', attrs = {'class':'t2'})[1]

    info_table = []

    for tr in cands_table.tbody.find_all('tr')[1:]:

        cands_info_dict = {
            'ПІБ':'',
            'Основні відомості':'',
            'Наявність(відсутність) заборгованості зі сплати аліментів':'',
            'Висування':'',
            'Передвиборна програма':'',
            'Дата реєстрації кандидатом':'',
            'Дата та підстава скасування реєстрації кандидата':'',
            'Округ':''
        }

        tds = [td for td in tr.findAll('td')]

        cands_info_dict['ПІБ'] = tds[0].text
        cands_info_dict['Основні відомості'] = tds[1].text
        cands_info_dict['Наявність(відсутність) заборгованості зі сплати аліментів'] = tds[2].text
        cands_info_dict['Висування'] = tds[3].text
        cands_info_dict['Дата реєстрації кандидатом'] = tds[5].text
        cands_info_dict['Дата та підстава скасування реєстрації кандидата'] = tds[6].text
        cands_info_dict['Округ'] = soup_okrug.find('p').text
        if tds[4].find('a') is not None:
            cands_info_dict['Передвиборна програма'] = 'https://www.cvk.gov.ua/pls/vnd2019/' + tds[4].find('a').get('href')

        info_table.append(cands_info_dict)
        time.sleep(1)

    return info_table

def get_csv_with_candidates(info_tables):
    cands_info = pd.concat(info_tables, ignore_index = True)
    cands_info = cands_info[cands_info.columns[::-1]]
    cands_info['Область'] = cands_info['Округ'].str.extract('\(([^)]+)')
    cands_info['Округ'] = cands_info['Округ'].str.extract('(\d+)')
    cands_info['ПІБ'] = [re.sub(r'([Є-ЯҐ]\w+)([Є-ЯҐ]\w+)', r'\1 \2', row) for row in cands_info['ПІБ']]
    return cands_info

def parse_candidates_programs(url):
    request = requests.get(url)
    request.encoding = 'UTF-8'
    soup_okrug = BeautifulSoup(request.content, 'lxml')
#     назва файлу з програмою - ПІБ депутата
    name = pd.read_html(str(soup_okrug),header=0)[5].iloc[:,0][0]
    tds = soup_okrug.find_all('td', {'class':'td3'})
    for td in tds:
        if td.a is not None:
            link = 'https://www.cvk.gov.ua/pls/vnd2019/' + td.a['href']
            response = requests.get(link)
            with open((PATH_TO_LOCATION_WITH_PROGRAMS + name), 'wb') as f:
                f.write(response.content)
            time.sleep(5)


In [ ]:
info_tables = []
cands_info = []

okrugs = get_links_for_okrugs()
for okrug in okrugs:
    info_tables.append(pd.DataFrame(parse_candidates_info(okrug)))
    # parse_candidates_programs(okrug)
cands_info = get_csv_with_candidates(info_tables)
cands_info.to_csv('output.csv', sep=',')

# data

## tried to get place of work

In [6]:
import re

In [1]:
import pandas as pd
df = pd.read_csv('output.csv')
df = df.drop('Unnamed: 0', axis=1)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import ipdb
def find_str(x):
#     ipdb.set_trace()
    for d in x:
        if ('осві' in d):
            return [ d for d in x if 'освіта' in x]
        else:
            return 'немає даних'

In [3]:
df['education'] = df['Основні відомості'].str.split(',').apply(lambda x: [ d for d in x if 'освіта' in d][0] )
df['place_of_residence'] = df['Основні відомості'].str.split(',').apply(lambda x: [ d for d in x if 'проживає в' in d][0] )

In [4]:
df['Основні відомості'][10]

'народився 6 березня 1975\xa0року в місті Первомайськ Миколаївської області, громадянин України, протягом останніх п’яти років проживає на території України, освіта вища, фізична особа – підприємець, член політичної партії Всеукраїнське об’єднання "Свобода", проживає в місті Вінниці, судимість відсутня, суб’єкт висування – політична партія Всеукраїнське об’єднання "Свобода".'

In [7]:
re.search(r'(?<=освіта вища,).*(?=, проживає в)', df['Основні відомості'][0]).group(0)


' доцент кафедри електричних станцій та систем, Вінницький національний технічний університет, безпартійнa'

In [9]:
df.education.value_counts()

 освіта вища                   2863
 освіта професійно-технічна     120
 освіта загальна середня        111
 освіта середня спеціальна       85
 освіта фахова передвища          1
Name: education, dtype: int64

In [10]:
def get_work(x):
    try:
        m = re.search(r'освіта.*(?=, проживає в)',x)
        return m.group(0)
    except:
        return 'no'

df['temp'] = df['Основні відомості'].apply(lambda x: get_work(x))
# df['temp'] = df.temp.str.split(',').str[2:].str.join(', ')

In [11]:
m = re.search(r'(?<=освіта вища,).*(?=, проживає в)', "народився 28 серпня 1975\xa0року в місті Вінниця, громадянин України, протягом останніх п’яти років проживає на території України, освіта вища, директор,Консалтинг експерт груп Діктум-Фактум ,безпартійний, проживає в місті Вінниці, судимість відсутня, самовисування.")
m

<_sre.SRE_Match object; span=(140, 201), match=' директор,Консалтинг експерт груп Діктум-Фактум ,>

In [12]:
df.temp.str.split(',').str[1:-1]

df['temp_2'] = df.temp.str.split(',').str[1:-1]
df['position'] = df.temp_2.str[0]
df['job_place'] = df.temp_2.str[1]

In [13]:
df['position'] = df.temp_2.str[0]
df['job_place'] = df.temp_2.str[1]

In [14]:
df

,Передвиборна програма,ПІБ,Основні відомості,Округ,Наявність(відсутність) заборгованості зі сплати аліментів,Дата та підстава скасування реєстрації кандидата,Дата реєстрації кандидатом,Висування,Область,education,place_of_residence,temp,temp_2,position,job_place
0,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Бевз Світлана Володимирівна,народилася 4 листопада 1973 року в місті Вінни...,11,-,,20.06.2019,Самовисування,Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, доцент кафедри електричних станці...",[ доцент кафедри електричних станцій та систем...,доцент кафедри електричних станцій та систем,Вінницький національний технічний університет
1,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Гижко Андрій Петрович,"народився 27 червня 1978 року в місті Вінниці,...",11,-,,24.06.2019,"Політична партія ""Європейська Солідарність""",Вінницька область,освіта вища,проживає в селі Березино Вінницького району В...,"освіта вища, перший заступник голови, Вінницьк...","[ перший заступник голови, Вінницька обласна ...",перший заступник голови,Вінницька обласна державна адміністрація
2,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Горлачов Ігор Ярославович,народився 31 березня 1984 року в місті Вінниці...,11,-,,24.06.2019,"Політична партія ""ОПОЗИЦІЙНА ПЛАТФОРМА – ЗА ЖИ...",Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, тимчасово не працює, член Політич...",[ тимчасово не працює],тимчасово не працює,NaN
3,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Давиденко Ганна Віталіївна,народилася 24 травня 1983 року в місті Вінниці...,11,-,,24.06.2019,Самовисування,Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, проректор з наукової та міжнародн...",[ проректор з наукової та міжнародної діяльнос...,проректор з наукової та міжнародної діяльності,"ВНЗ ""Відкритий міжнародний університет розвит..."
4,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Кістіон Володимир Євсевійович,народився 31 травня 1965 року в селі Довжок Ям...,11,-,,12.06.2019,Самовисування,Вінницька область,освіта вища,проживає в місті Києві,"освіта вища, Віце-прем’єр-міністр України, без...",[ Віце-прем’єр-міністр України],Віце-прем’єр-міністр України,NaN
5,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Ковальов Андрій Євгенійович,"народився 24 сiчня 1977 року в місті Вінниці, ...",11,-,,25.06.2019,"ПОЛІТИЧНА ПАРТІЯ ""СИЛА І ЧЕСТЬ""",Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, радник генерального директора, АТ...","[ радник генерального директора, АТ ""Вінницяо...",радник генерального директора,"АТ ""Вінницяобленерго"""
6,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Кудлаєнко Сергій Володимирович,"народився 13 жовтня 1975 року в місті Вінниці,...",11,-,,24.06.2019,Самовисування,Вінницька область,освіта вища,проживає в селі Агрономічне Вінницького район...,"освіта вища, доцент кафедри менеджменту та адм...",[ доцент кафедри менеджменту та адмініструванн...,доцент кафедри менеджменту та адміністрування,Вінницький торговельно-економічний інститут К...
7,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Лепей Олена Володимирівна,"народилася 3 жовтня 1977 року в місті Вінниці,...",11,-,,25.06.2019,"Політична партія ""ОПОЗИЦІЙНИЙ БЛОК""",Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, завідувач, Вінницьке відділення н...","[ завідувач, Вінницьке відділення навчально-н...",завідувач,Вінницьке відділення навчально-наукового інст...
8,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Пашковський Максим Ігорович,народився 24 березня 1983 року в місті Вінниці...,11,-,,25.06.2019,"ПОЛІТИЧНА ПАРТІЯ ""СЛУГА НАРОДУ""",Вінницька область,освіта вища,проживає в місті Вінниці,"освіта вища, директор, ТОВ ""Панчер Солюшнс"", б...","[ директор, ТОВ ""Панчер Солюшнс""]",директор,"ТОВ ""Панчер Солюшнс"""
9,https://www.cvk.gov.ua/pls/vnd2019/showdocpf72...,Приходько Анастасія Костянтинівна,"народилася 21 квiтня 1987 року в місті Києві, ...",11,-,,24.06.2019,"політична партія Всеукр

## connecting elections data with previous presidentail elections

In [1]:
import pandas as pd

In [32]:
combined_2010_2012_elections = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSNxiVppRf_pyTzKIXI-NA-5ncgsmjr8EmKSNUlarYzpdVC12ToK80zDjDXRkWvw_lmF6ng-yoTRMvP/pub?gid=0&single=true&output=csv')

In [258]:
import requests

In [310]:
response = requests.get('https://www.cvk.gov.ua/pls/vp2010/wp335f0d8.html?PT001F01=700')

In [311]:
soup = bs4.BeautifulSoup(response.content)

In [313]:
a = ['https://www.cvk.gov.ua/pls/vp2010/' + d['href'] for d in soup.select('a.a1')]
b = [d.text.strip() for d in soup.select('td.td3')]

In [331]:
re = requests.get(a[0])
pd.read_html(re.content)[2].iloc[0,:]

0                     Дільн.
1      1. Кількість виборців
2            2. Взяли участь
3     3. Недійсних бюлетенів
4               Богословська
5                  Бродський
6                   Гриценко
7                   Костенко
8                     Литвин
9                      Мороз
10                     Пабат
11                 Противсіх
12                  Ратушняк
13                  Рябоконь
14                 Симоненко
15                    Супрун
16                 Тимошенко
17                   Тігіпко
18                  Тягнибок
19                    Ющенко
20                  Янукович
21                    Яценюк
22      4. Не підтр. жодного
Name: 0, dtype: object

In [337]:
collections = []

for d in a:
    for dd in b:
        re = requests.get(d)
        table = pd.read_html(re.content)[2]
        table.columns = table.iloc[0,:]
        table = table.iloc[1:,:]
        
        collections.append(pd.DataFrame(table))

KeyboardInterrupt: 

In [335]:
collections

,0
0,Дільн.
1,1. Кількість виборців
2,2. Взяли участь
3,3. Недійсних бюлетенів
4,Богословська
5,Бродський
6,Гриценко
7,Костенко
8,Литвин
9,Мороз
